# Project 3 : Named Entity Disambiguation

## Goal : 
**Map entities to Wikipedia URL**

## Data loading and description

In [55]:
import pandas as pd
data_path = "/kaggle/input/dis-project-3-named-entity-disambiguation/"
train = pd.read_table(data_path+"train.csv", sep=",", index_col='id')
test = pd.read_table(data_path+"test.csv", sep=",", index_col='id')
submission = pd.read_table(data_path+"sample_submission.csv", sep=",", index_col='id')
display("Train : ", train)
display("Test : ", test)
display("Submission : ", submission)

'Train : '

,token,entity_tag,full_mention,wiki_url
id,,,,
1,-DOCSTART- (1 EU),NaN,NaN,NaN
2,EU,B,EU,--NME--
3,rejects,NaN,NaN,NaN
4,German,B,German,http://en.wikipedia.org/wiki/Germany
5,call,NaN,NaN,NaN
...,...,...,...,...
218501,Swansea,B,Swansea,http://en.wikipedia.org/wiki/Swansea_City_A.F.C.
218502,1,NaN,NaN,NaN
218503,Lincoln,B,Lincoln,http://en.wikipedia.org/wiki/Lincoln_City_F.C.


'Test : '

,token,entity_tag,full_mention,wiki_url
id,,,,
0,-DOCSTART- (947testa CRICKET),NaN,NaN,NaN
1,CRICKET,NaN,NaN,NaN
2,-,NaN,NaN,NaN
3,LEICESTERSHIRE,B,LEICESTERSHIRE,?
4,TAKE,NaN,NaN,NaN
...,...,...,...,...
104885,brother,NaN,NaN,NaN
104886,",",NaN,NaN,NaN
104887,Bobby,B,Bobby,?


'Submission\xa0: '

,wiki_url
id,
0,NOT_FOUND
1,NOT_FOUND
2,NOT_FOUND
3,PREDICTED_WIKI_LINK
4,NOT_FOUND
5,NOT_FOUND
6,NOT_FOUND
7,NOT_FOUND
8,NOT_FOUND


In [2]:
wiki_items = pd.read_table(data_path+"wiki_lite/wiki_items.csv", sep="," )
redirects = pd.read_table(data_path+"wiki_lite/enwiki_redirects.tsv", sep="\t", header=None)
item_aliases = pd.read_table(data_path+"wiki_lite/item_aliases.csv", sep=",")
display("Wiki_items : ", wiki_items)
display("redirects : ", redirects)
display("item_aliases : ", item_aliases)

'Wiki_items : '

,item_id,en_label,en_description,wikipedia_title
0,1,Universe,totality of space and all contents,Universe
1,2,Earth,third planet from the Sun in the Solar System,Earth
2,3,life,matter capable of extracting energy from the e...,Life
3,4,death,permanent cessation of vital functions,Death
4,5,human,"common name of Homo sapiens, unique extant spe...",Human
...,...,...,...,...
5216231,77042017,HR 4523,NaN,HD 102365
5216232,77043280,Charlie Johnston,NaN,Charlie Johnstone
5216233,77231860,Aldo Rossi,musician,Aldo Rossi (musician)
5216234,77240068,Ebenezer Baptist Church,"church in Atlanta, Georgia, USA",Ebenezer Baptist Church


'redirects : '

,0,1
0,!,Exclamation mark
1,! (CONFIG.SYS directive),CONFIG.SYS
2,! (Donnie Vie Album),Donnie Vie
3,! (Donnie Vie album),Donnie Vie
4,! (The Song Formerly Known As),Unit (album)
...,...,...
8558898,󠁽,Tags (Unicode block)
8558899,󠁾,Tags (Unicode block)
8558900,󠁿,Tags (Unicode block)
8558901,􍁷,Private Use Areas


'item_aliases : '

,item_id,en_alias
0,1,Our Universe
1,1,The Universe
2,1,The Cosmos
3,1,cosmos
4,2,Blue Planet
...,...,...
1495310,76858465,Photinus luciferin 4-monooxygenase (adenosine ...
1495311,76858465,firefly luciferase
1495312,76868858,12 Canum Venaticorum
1495313,76868858,Alpha Canum Venaticorum


In [3]:
statements = pd.read_table(data_path+"wiki_lite/statements.csv", sep=",")
properties = pd.read_table(data_path+"wiki_lite/property.csv", sep=",")
display("Properties : ", properties)
display("Statements : ", statements)

'Properties : '

,property_id,en_label,en_description
0,6,head of government,"head of the executive power of this town, city..."
1,10,video,"relevant video. For images, use the property P..."
2,14,traffic sign,"graphic symbol describing the item, used at th..."
3,15,route map,image of route map at Wikimedia Commons
4,16,highway system,system (or specific country specific road type...
...,...,...,...
6980,7663,Scienza a due voci ID,identifier for an Italian female scientist in ...
6981,7665,FMV World ID,identifier of a video game in the FMV World da...
6982,7666,Visuotinė lietuvių enciklopedija ID,identifier for an item in the online edition o...
6983,7667,Hellenic Civil Aviation Authority airport code,NaN


'Statements : '

,source_item_id,edge_property_id,target_item_id
0,1,398,497745
1,1,398,1133705
2,1,398,1139177
3,1,398,7439451
4,1,398,15241043
...,...,...,...
26903183,77240068,31,16970
26903184,77240068,131,23556
26903185,77242291,17,145
26903186,77242291,131,23311


In [45]:
# Fullmention to en_label (wiki_item)
def find_url(full_mention, wiki_items, en_label, item_aliases, en_alias, redirects):
    full_mention = str(full_mention)
    if full_mention in en_label:
        title = wiki_items[wiki_items.en_label == full_mention].wikipedia_title.iloc[0]
    elif full_mention in en_alias:
        item_id = item_aliases[item_aliases.en_alias == full_mention].item_id.iloc[0]
        title = wiki_items[wiki_items.item_id == item_id].wikipedia_title.iloc[0]
    else:
        return "NOT_FOUND"
    
    if title in redirects[0]:
        title = redirects[redirects[0] == title][1].iloc[0]

    # Underscore 
    title = title.replace(" ","_")

    url = 'http://en.wikipedia.org/wiki/'+title
    return url

def compute_urls(test_,  wiki_items, en_label, item_aliases, en_alias, redirects):
    print(test_)
    
    results = []
    for i in range(len(test_)):
        full_mention = test_.iloc[i].full_mention
        results.append(find_url(full_mention, wiki_items, en_label, item_aliases, en_alias, redirects))
    return results

en_label = list(set(wiki_items.en_label.to_list()))
en_alias = list(set(item_aliases.en_alias.to_list()))

In [5]:
find_url("EU", wiki_items, en_label, item_aliases, en_alias, redirects)

'http://en.wikipedia.org/wiki/EU_(group)'

In [ ]:
%%time
test_ = test[test.wiki_url == "?"]
urls = compute_urls(test_,  wiki_items, en_label, item_aliases, en_alias, redirects)

In [56]:
test.loc[test['wiki_url'] != "?", "wiki_url"] = "NOT_FOUND"
test.loc[test['wiki_url'] == "?", 'wiki_url']= urls
test

,token,entity_tag,full_mention,wiki_url
id,,,,
0,-DOCSTART- (947testa CRICKET),NaN,NaN,NOT_FOUND
1,CRICKET,NaN,NaN,NOT_FOUND
2,-,NaN,NaN,NOT_FOUND
3,LEICESTERSHIRE,B,LEICESTERSHIRE,NOT_FOUND
4,TAKE,NaN,NaN,NOT_FOUND
...,...,...,...,...
104885,brother,NaN,NaN,NOT_FOUND
104886,",",NaN,NaN,NOT_FOUND
104887,Bobby,B,Bobby,http://en.wikipedia.org/wiki/Bobby


In [61]:
test['wiki_url'].to_csv("/kaggle/working/submission.csv")